In [1]:
MUSIC_CSV_PATH = 'D:/Works/workspace/text2music/csvfiles/musiccaps-public.csv'
AUDIOS_DIR = 'D:/Works/workspace/text2music/audios'
URL_BASE = 'https://www.youtube.com/watch?v='

In [12]:
import csv
import subprocess
import os
from pathlib import Path

AUDIOS_DIR = "downloaded_audios"
URL_BASE = "https://www.youtube.com/watch?v="

def download_clip(
    video_identifier,
    output_filename,
    start_time,
    end_time,
    tmp_dir=AUDIOS_DIR,
    num_attempts=1,
    url_base=URL_BASE
):
    status = False
    command = f"""
        yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" "{url_base}{video_identifier}"
    """.strip()
    
    attempts = 0
    while attempts < num_attempts:
        try:
            result = subprocess.run(command, shell=True, check=True, capture_output=True)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output.decode()
        else:
            break

    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    return status, 'Downloaded' if status else 'Failed to download'

def process(example, data_dir):
    outfile_path = str(data_dir / f"{example['ytid']}.wav")
    status = True
    if not os.path.exists(outfile_path):
        status, log = download_clip(
            example['ytid'],
            outfile_path,
            example['start_s'],
            example['end_s'],
        )

    example['audio'] = outfile_path
    example['download_status'] = status
    return example

def read_csv_and_process(file_path, data_dir):
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True)

    with open(file_path, mode='r') as csv_file:
        reader = csv.DictReader(csv_file)
        examples = [row for row in reader]

    processed_examples = []
    for example in examples:
        # Convert start_s and end_s to integers
        example['start_s'] = int(example['start_s'])
        example['end_s'] = int(example['end_s'])
        processed_example = process(example, data_dir)
        processed_examples.append(processed_example)
        print(f"Processed {processed_example['ytid']}: {processed_example['download_status']}")

    print("Download and extraction complete.")
    return processed_examples

# Usage
csv_file_path = "D:/Works/workspace/text2music/csvfiles/musiccaps-public.csv"  # Replace with your CSV file path
read_csv_and_process(csv_file_path, AUDIOS_DIR)


KeyboardInterrupt: 

In [2]:
import pandas as pd

df = pd.read_csv(MUSIC_CSV_PATH, index_col = False)

In [3]:
# JUST HELPER METHODS IN THIS CELL 

import subprocess
import os
from pathlib import Path
from datasets import Audio


def download_clip(
    video_identifier,
    output_filename,
    start_time,
    end_time,
    data_dir=AUDIOS_DIR,
    num_attempts=1,
    url_base=URL_BASE
):
    status = False

    command = f"""
        yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" "{url_base}{video_identifier}" --force-keyframes-at-cuts
    """.strip()


    
    attempts = 0
    while attempts < num_attempts:
        try:
            output = subprocess.check_output(command, shell=True,
                                                stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output.decode()  
        else:
            break

    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    return status, 'Downloaded' if status else 'Failed to download'

def process(example):
    outfile_path = str(data_dir / f"{example['ytid']}.wav")
    status = True
    if not os.path.exists(outfile_path):
        status = False
        status, log = download_clip(
            example['ytid'],
            outfile_path,
            example['start_s'],
            example['end_s'],
        )

    example['audio'] = outfile_path
    example['download_status'] = status
    return example

In [4]:
df.sample(5)


,ytid,start_s,end_s,audioset_positive_labels,aspect_list,caption,author_id,is_balanced_subset,is_audioset_eval
2735,S2yXmXUJ5xU,30,40,"/m/0326g,/m/0342h,/m/042v_gx,/m/04rlf,/m/04szw...","['flamenco', 'no singer', 'instrumental', 'sin...",This is a flamenco music piece. The only instr...,9,False,False
2368,NqDxpJ2uR_8,0,10,"/m/04rlf,/m/0cj0r","['pop', 'wave sounds', 'digital sound effects'...",Digital sound effects are rising in a high pit...,6,False,True
4808,qie2k2gZ7wA,490,500,"/m/04rlf,/m/07sr1lc,/t/dd00126","['low quality audio recording', 'rock genre', ...",The rock music features a screaming male voice...,2,True,True
410,2uagA3ujRtM,0,10,"/m/02yds9,/m/04rlf","['purring sound', 'violins', 'no percussion', ...",This clip features purring sounds as the main ...,0,False,True
5059,tk4s-nlhmEQ,30,40,"/m/04rlf,/t/dd00002","['poor audio quality', 'baby crying', 'horn pl...",A baby cries in pain followed by a slowed down...,1,True,True


In [5]:
df.columns

Index(['ytid', 'start_s', 'end_s', 'audioset_positive_labels', 'aspect_list',
       'caption', 'author_id', 'is_balanced_subset', 'is_audioset_eval'],
      dtype='object')

In [8]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5521 entries, 0 to 5520
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ytid                      5521 non-null   object
 1   start_s                   5521 non-null   int64 
 2   end_s                     5521 non-null   int64 
 3   audioset_positive_labels  5521 non-null   object
 4   aspect_list               5521 non-null   object
 5   caption                   5521 non-null   object
 6   author_id                 5521 non-null   int64 
 7   is_balanced_subset        5521 non-null   bool  
 8   is_audioset_eval          5521 non-null   bool  
dtypes: bool(2), int64(3), object(4)
memory usage: 312.8+ KB
None


In [23]:
def read_csv_and_process(MUSIC_CSV_PATH, data_dir):
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True)
    
    for row in df.itertuples():
        video_identifier = row.ytid
        output_filename = str(MUSIC_CSV_PATH/ f"{video_identifier}.wav")
        start_time = row.start_s
        end_time = row.end_s
        status, message = download_clip(video_identifier, output_filename, start_time, end_time)
        print(f"Processed {video_identifier}: {message}")

    print("Download and extraction complete.")

In [32]:
import csv
import subprocess
import os

# Function to download audio segment directly in wav format
def download_audio_segment(ytid, start_s, end_s, output_path):
    url = f'https://www.youtube.com/watch?v={ytid}'
    output_file = f'{output_path}/{ytid}.wav'
    duration = end_s - start_s  # Calculate the duration of the segment
    
    # Specify the path to the directory containing FFmpeg executable
    ffmpeg_location = "C:/ffmpeg"

    command = [
        'yt-dlp',
        '--extract-audio', '--audio-format', 'wav',  # Extract audio and convert to WAV
        '--postprocessor-args', f'ffmpeg:-ss {start_s} -t {duration}',  # Specify start time and duration
        '--ffmpeg-location', ffmpeg_location,  # Specify the path to FFmpeg
        '-o', output_file,  # Output file
        url
    ]
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"Successfully downloaded {ytid}.wav")
        print(result.stdout)
        print(result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred for video {ytid}: {e.stderr}")

# Directory to save files
output_dir = 'AUDIOS_DIR'
os.makedirs(output_dir, exist_ok=True)

# Sample CSV data
csv_data = """ytid,start_s,end_s,audioset_positive_labels,aspect_list,caption,author_id,is_balanced_subset,is_audioset_eval
e_W17jp40G4,30,40,/m/015vgc,/m/01c194,/m/04rlf,['low quality', 'wooden percussive element', '...	The low quality recording features a passionat...	4	True	True
R4jlQEweREY,30,40,/m/02bxd,/m/04rlf,['didgeridoo', 'solo instrumentalist', 'percus...	The song is an instrumental. The song is mediu...	1	False	True
Db0c4aAwUbY,30,40,/m/04rlf,/m/0l156b,['funk', 'steeldrums', 'acoustic drums', 'cong...	A lot of people are playing different parts of...	6	False	True
T9dKp1EN4p8,30,40,/m/02bxd,/m/02cz_7,/m/04rlf,['low quality', 'didgeridoo solo melody', 'low...	The low quality recording features a low didge...	4	False	True
RDCXvJPzT5k,520,530,/m/085jw,/m/0l14l2,['home recording', 'conch shell', 'horn-like s...	This is a home recording of a conch shell bein...	9	False	False"""

# Parse the CSV data
reader = csv.DictReader(csv_data.splitlines())
for row in reader:
    ytid = row['ytid']
    start_s = int(row['start_s'])
    end_s = int(row['end_s'])

    # Download audio segment directly in wav format
    download_audio_segment(ytid, start_s, end_s, output_dir)

print("Download and extraction complete.")


Error occurred for video e_W17jp40G4: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location



KeyboardInterrupt: 